In [1]:
# Optional helpers: make JSON-like names safe if accidentally typed unquoted in a cell
try:
    null
except NameError:
    null = None
try:
    true
except NameError:
    true = True
try:
    false
except NameError:
    false = False



In [27]:
from pathlib import Path
import uuid
import re

ALLOWED_EXTENSIONS = {
    ".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".tif", ".webp", ".heic", ".heif"
}


def prompt_for_directory() -> Path:
    folder_input = input("Enter the path to the folder containing images: ").strip().strip('"').strip("'")
    folder_path = Path(folder_input).expanduser().resolve()
    if not folder_path.exists():
        raise FileNotFoundError(f"Folder does not exist: {folder_path}")
    if not folder_path.is_dir():
        raise NotADirectoryError(f"Not a directory: {folder_path}")
    return folder_path


def prompt_for_prefix(default: str = "SMALL_z") -> str:
    raw = input(f"Enter the prefix for new names (default '{default}'): ").strip()
    # Treat JSON-like null/none as empty
    if raw.lower() in {"null", "none"}:
        raw = ""
    value = raw or default
    # Sanitize for Windows-incompatible characters
    value = re.sub(r"[\\\\/:*?\"<>|]", "_", value)
    value = value.strip(" ") or default
    return value


def find_image_files(folder: Path):
    image_paths = [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in ALLOWED_EXTENSIONS]
    # Sort deterministically by filename
    image_paths.sort(key=lambda p: p.name.lower())
    return image_paths


def rename_images_sequentially(folder: Path, prefix: str):
    image_files = find_image_files(folder)
    if not image_files:
        print("No image files found to rename.")
        return

    print(f"Found {len(image_files)} image(s). Renaming to {prefix}_0..{prefix}_{len(image_files) - 1} while preserving extensions...")

    temp_files = []
    # Stage 1: rename everything to unique temporary names to avoid collisions
    for original_path in image_files:
        temp_name = f"{original_path.stem}__TMP_RENAME__{uuid.uuid4().hex}{original_path.suffix}"
        temp_path = original_path.with_name(temp_name)
        original_path.rename(temp_path)
        temp_files.append(temp_path)

    # Stage 2: rename temps to final names
    renamed_paths = []
    try:
        for index, temp_path in enumerate(temp_files):
            final_name = f"{prefix}_{index}{temp_path.suffix}"
            final_path = folder / final_name
            if final_path.exists():
                raise FileExistsError(f"Final target already exists: {final_path}")
            temp_path.rename(final_path)
            renamed_paths.append(final_path)
    except Exception as exc:
        print(f"Error during renaming: {exc}. Attempting best-effort rollback...")
        # Best-effort rollback: try to remove the TMP marker from any remaining temp files
        for path in folder.iterdir():
            if path.is_file() and "__TMP_RENAME__" in path.name:
                cleaned_name = path.name.replace("__TMP_RENAME__", "")
                try:
                    path.rename(folder / cleaned_name)
                except Exception:
                    pass
        raise

    print(f"Renamed {len(renamed_paths)} image(s) in: {folder}")
    for preview in renamed_paths[:10]:
        print(f"- {preview.name}")
    if len(renamed_paths) > 10:
        print("...")


if __name__ == "__main__":
    target = prompt_for_directory()
    prefix = prompt_for_prefix("SMALL_z")
    rename_images_sequentially(target, prefix)



Found 100 image(s). Renaming to SMALL_z_0..SMALL_z_99 while preserving extensions...
Renamed 100 image(s) in: D:\Faisal sir Dataset-20250809T142456Z-1-001\Faisal sir Dataset\Small\z
- SMALL_z_0.png
- SMALL_z_1.png
- SMALL_z_2.png
- SMALL_z_3.png
- SMALL_z_4.png
- SMALL_z_5.png
- SMALL_z_6.png
- SMALL_z_7.png
- SMALL_z_8.png
- SMALL_z_9.png
...
